In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon as Matpoly
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree

def get_tree_poly(x, y, deg):
    coords = [(0.0, 0.8), (0.125, 0.5), (0.0625, 0.5), (0.2, 0.25), (0.1, 0.25),
              (0.35, 0.0), (0.075, 0.0), (0.075, -0.2), (-0.075, -0.2),
              (-0.075, 0.0), (-0.35, 0.0), (-0.1, 0.25), (-0.2, 0.25),
              (-0.0625, 0.5), (-0.125, 0.5)]
    poly = Polygon(coords)
    return affinity.translate(affinity.rotate(poly, deg, origin=(0, 0)), x, y)

def pack_optimized(n):
    # Aggressive interlocking constants
    dx, dy = 0.462, 0.522
    ox = 0.231 
    buffer = 0.0
    
    while True:
        trees = []
        cols = int(np.ceil(np.sqrt(n)))
        for i in range(n):
            r, c = divmod(i, cols)
            flip = (i % 2 != 0)
            tx = c * (dx + buffer) + (ox if flip else 0)
            ty = r * (dy + buffer)
            ang = 180 if flip else 0
            trees.append({'x': tx, 'y': ty, 'deg': ang, 'poly': get_tree_poly(tx, ty, ang)})
        
        polys = [t['poly'] for t in trees]
        idx_tree = STRtree(polys)
        overlap = False
        for j, p in enumerate(polys):
            for k in idx_tree.query(p):
                if k != j and p.intersects(polys[k]):
                    overlap = True; break
            if overlap: break
        
        if not overlap: return trees
        buffer += 0.005 

# 1. GENERATE SUBMISSION
all_data = []
for n in range(1, 201):
    res = pack_optimized(n)
    for i, t in enumerate(res):
        all_data.append({
            'id': f"{n:03d}_{i}",
            'x': f"s{t['x']:.12f}", 'y': f"s{t['y']:.12f}", 'deg': f"s{t['deg']}.0"
        })
pd.DataFrame(all_data).to_csv("submission.csv", index=False)
print("✅ Submission Saved.")

# 2. VISUAL AUDIT (Plotting Shipment 200)
def plot_mosaic(trees):
    fig, ax = plt.subplots(figsize=(10, 10))
    for t in trees:
        x, y = t['poly'].exterior.xy
        # Dark green for 0 deg, Lime green for 180 deg
        color = '#1B5E20' if t['deg'] == 0 else '#8BC34A'
        ax.add_patch(Matpoly(list(zip(x, y)), facecolor=color, edgecolor='w', lw=0.2))
    ax.set_aspect('equal')
    ax.autoscale_view()
    plt.title("The Dimer Mosaic: Interlocking Efficiency Audit")
    plt.axis('off')
    plt.show()

print("🎨 Generating Visualization for Shipment 200...")
plot_mosaic(pack_optimized(200))